<a href="https://colab.research.google.com/github/Zeffar/Machine-Learning/blob/main/MNIST_digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please run the following code cells in order :)  (Ctrl+F9 to run all cells)


To see the code, double-click a cell.


In [ ]:
#@title Import Modules and Data
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# improve formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)

#import data set
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

#Normalise the Data
x_train_normalized = x_train/255
x_test_normalized = x_test/255

In [ ]:
#@title Define the plotting function { display-mode: "both" }
def plot_curve(epochs, hist, list_of_metrics):
  """Plot a curve of one or more classification metrics vs. epoch."""  
  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()
  print("done")

In [ ]:
#@title Create and train the model
#create model class
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Convolution2D
from tensorflow.keras.optimizers import Adam

def create_model(my_learning_rate):
  model = Sequential()

  #turn 28x28 into 784x1 array
  model.add(Flatten())

  #hidden layers
  model.add(Dense(units=256, activation='relu'))
  model.add(Dense(units=128, activation='relu'))
  
  #dropout layer 
  model.add(Dropout(rate=0.4))

  #output layer
  model.add(Dense(units=10, activation='softmax'))     

  model.compile(optimizer=Adam(lr=my_learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  
  return model    


#train model class
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):
  """Train the model by feeding it data."""

  #checpoint
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath='savedModelWeighs', period=10, save_weights_only=True )

  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, callbacks=[model_checkpoint_callback], shuffle=True, 
                      validation_split=validation_split)
 
  #metrics snapshot at each epoch
  epochs = history.epoch
  hist = pd.DataFrame(history.history)

  return epochs, hist  

In [ ]:
#@title Train the model (this takes ~1 minute)
#hyperparameters
learning_rate = 0.003
epochs = 50
batch_size = 4000
validation_split = 0.2

#topography
my_model = create_model(learning_rate)

#train the model on the set hyperpar.
epochs, hist = train_model(my_model, x_train_normalized, y_train, 
                           epochs, batch_size, validation_split)

#plot metric x epoch
list_of_metrics_to_plot = ['accuracy']
plot_curve(epochs, hist, list_of_metrics_to_plot)

#evaluate the model
print("\n Evaluate the new model against the test set:")
my_model.evaluate(x=x_test_normalized, y=y_test, batch_size=batch_size)

The model has an overall accuracy of ~98% over the test data

In [ ]:
#@title Manually test the model: following code outputs the predicted value for the chosen test case, followed by the visualisation of said test case
test_value=894 #edit this value
plt.imshow(x_test[test_value],cmap='gray', vmin=0, vmax=255)
pred=loadedModel.predict(x_test)
print("predicted value is:",np.argmax(pred[test_value]))